# SoftVC Acoustic Model
[![Generic badge](https://img.shields.io/badge/GitHub-softVCam-9cf.svg)][github]
[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)][notebook]

Author: [tarepan]

[github]:https://github.com/tarepan/softVC_AM
[notebook]:https://colab.research.google.com/github/tarepan/softVC_AM/blob/main/softVC_AM.ipynb
[tarepan]:https://github.com/tarepan

## Colab Check
Check
- Google Colaboratory runnning time
- GPU type
- Python version
- CUDA version

In [ ]:
!cat /proc/uptime | awk '{print $1 /60 /60 /24 "days (" $1 "sec)"}'
!head -n 1 /proc/driver/nvidia/gpus/**/information
!python --version
!pip show torch | sed '2!d'
!/usr/local/cuda/bin/nvcc --version | sed '4!d'

## Setup

Install the package from `tarepan/softVC_AM` public repository

In [ ]:
# GoogleDrive
from google.colab import drive
drive.mount('/content/gdrive')

# Dedicated dependencies install
# !pip install "torch==1.10.0" -q      # Based on your PyTorch environment
# !pip install "torchaudio==0.10.0" -q # Based on your PyTorch environment

!git clone https://github.com/tarepan/softVC_AM
%cd softVC_AM
# repository install
# !pip uninstall softvcam -y -q
# !pip install git+https://github.com/tarepan/softVC_AM -q

!pip install git+https://github.com/tarepan/softVC_hubert
!pip install git+https://github.com/tarepan/speechcorpusy.git

## Training

### Step 0: Data

In [ ]:
import speechcorpusy

corpus = speechcorpusy.load_preset("LJ", root="/content/gdrive/MyDrive/ML_data")
corpus.get_contents()


### Step 1: Preprocessing

In [ ]:
!mkdir data_softVC
!mkdir data_softVC/train

#### Data paths
Make list of wav file path relative to data root, under `./data_softVC/train`.

In [ ]:
from pathlib import Path
import speechcorpusy

in_wav_dir = Path("tmp/corpuses/LJSpeech")
corpus = speechcorpusy.load_preset("LJ", root="/content/gdrive/MyDrive/ML_data")
all_utterances = corpus.get_identities()
uttrs_train, uttrs_val = all_utterances[:-20], all_utterances[-20:]

with open("data_softVC/train/train.txt", "a", encoding="utf-8") as f:
    for item in uttrs_train:
        path_str = str(corpus.get_item_path(item).relative_to(in_wav_dir))
        f.write(path_str+"\n")

with open("data_softVC/train/validation.txt", "a", encoding="utf-8") as f:
    for item in uttrs_val:
        path_str = str(corpus.get_item_path(item).relative_to(in_wav_dir))
        f.write(path_str+"\n")


#### Wave-to-Unit

In [ ]:
from pathlib import Path
from hubert.encode import encode_dataset


#### Change here for your data ###########################
soft_disc = "soft"
# soft_disc = "discrete"
in_wav_dir = Path("tmp/corpuses/LJSpeech")    # Directory containing .wav input files
##########################################################

out_unit_dir = Path(f"./data_softVC/train/{soft_disc}") # Directory in which new unit series will be saved
encode_dataset(soft_disc, in_wav_dir, out_unit_dir, ".wav")


#### Wave-to-Mel

In [ ]:
!python mels.py "tmp/corpuses/LJSpeech" "./data_softVC/train/mels"

In [ ]:
# Launch TensorBoard
%load_ext tensorboard
%tensorboard --logdir /content/gdrive/MyDrive/ML_results/softVC_AM

!python train.py data_softVC/train /content/gdrive/MyDrive/ML_results/softVC_AM
# !python train.py data_softVC/train /content/gdrive/MyDrive/ML_results/softVC_AM --resume=/content/gdrive/MyDrive/ML_results/softVC_AM/... # Resume


## Inference

### Unit-to-Mel

#### Step 0: Wave-to-Unit
If you do NOT have unit .npy, first generate it.

In [ ]:
!pip install git+https://github.com/tarepan/softVC_hubert

In [ ]:
from pathlib import Path
from hubert.encode import encode_dataset


#### Change here for your data ###########################
in_wav_dir = Path("./data_softVC/wavs")    # Directory containing .wav input files
out_unit_dir = Path("./data_softVC/units") # Directory in which new unit series will be saved
##########################################################

out_unit_dir.mkdir(parents=True, exist_ok=True)
encode_dataset("soft", in_wav_dir, out_unit_dir, ".wav")


#### Step 1: Unit-to-Mel

In [ ]:
!python ./generate.py soft <in-unit-dir> <out-mel-dir>
# !python ./generate.py soft "./data_softVC/units" "./data_softVC/mspcs" # For example